# Submission of Calculation with Subsequent Results Query

In this demo, we will submit a calculation, check the status of the job and query the results after it is finished.
Our initial geometry is read from an xyz file.

## Initialization of Workspace 

Specify the Azure quantum workspace and the 'connection string' which allows us to connect to the workspace

In [ ]:
from azure.quantum import Workspace
from azure.quantum.job import JobFailedWithResultsError

# insert connection string from Azure Portal Workspace Access Keys
connection_string = ""
workspace = Workspace.from_connection_string(connection_string)

In [ ]:
# To submit Accelerated DFT jobs, we will be using the microsoft.dft target in the workspace.
print("Verifying access to Accelerated DFT target.")
target = workspace.get_targets("microsoft.dft")
print("Verification complete.")

This example is an form of FL172, which contains an Ru atom

In [ ]:
# First, let's define the molecular structure, loaded from an xyz file.
from pathlib import Path
GeomFile = "molecules/FL172.xyz"

In [ ]:
# For visualization of molecules and orbitals:
import py3Dmol

data = None
with open(GeomFile, "r") as infile:
    data = infile.read()

view = py3Dmol.view()
view.addModel(data)
view.setStyle({"stick": {}})
# if you prefer ball and stick molecules:
#view.setStyle({"stick":{},"sphere": {"scale":0.25}})
view.show()

In [ ]:
# Secondly, let's give a name for the job.
job_name = 'FL172_spe'

Now we submit the calculations to MADFT service.

In [ ]:
# Next, we create a dictionary variable to specify the parameters for the DFT calculation.

# The def2-tzvpp basis is large and for Ru contains up to g functions.

dft_input_params = {
  "tasks": [
    {
      "taskType": "spe",
      "basisSet": { "name": 'def2-tzvpp'}, 
      "xcFunctional": { "name": "m06-2x", "gridLevel": 4 },
      "molecule": { "charge": 0, "multiplicity": 1 },
      "scf":{"method":"rks","dispersion":"d3zero","convergeThreshold":1e-8, "requireWaveFunction": True}
    }
  ]
}

# We are now ready to submit the Job using the target.submit call. It takes three parameters-
# 1. The input molecule in xyz format.
# 2. The DFT parameters that we declared above.
# 3. A friendly name to help identify the job in the Azure Portal later.

print("Submitting DFT job.")

job = target.submit(
    input_data=Path(GeomFile).read_text(),
    input_params = dft_input_params,
    name= job_name)
    
print("\nDFT job has been submitted.")
print(f"\nJob name: {job_name}")


Show the status of the job. If the job has finished, read the results of the job

In [ ]:
job.refresh()
print(f'Job: "{job_name}" is {job.details.status}')
if job.details.status == 'Succeeded':
    qcschema = job.get_results()["results"][0]

The results of the calculation are stored in the QCSchema format dict.

For an SPE calculation we can see the energy by simply looking at the key "return_result".
(For SPF calculations, this key returns the force)

In [ ]:
print("SPE Result: ",qcschema["return_result"])

In [ ]:
print("Number of Basis Functions: ", qcschema["properties"]["calcinfo_nbasis"])
print("Total Energy (Hartree): ", qcschema["properties"]["return_energy"])
print("Nuclear Repulsion Energy (Hartree): ", qcschema["properties"]["nuclear_repulsion_energy"])
print("Total Calculation Time (s): ", qcschema["provenance"]["total_time_seconds"])

The output can be explored using qcschema.keys()

Wavefunction information is also saved in the output if "requireWaveFunction": True was set.
The "wavefunction" key contains orbitals, orbital energies, orbital occupancies, and Fock matrices.
This will be used in our later examples for property calculations.

## Output to QCSchema json file

In [ ]:
import json
qcschema_json = job_name + "_output.json"
with open(qcschema_json, "w") as fp:
    json.dump(qcschema, fp)